In [135]:
import warnings
import yaml

from src.validation import make_cross_validation

from tqdm import tqdm
from typing import List, Tuple, Optional

import os 
import numpy as np
import pandas as pd
import seaborn as sns
import catboost as cb
import matplotlib.pyplot as plt
import missingno as msno
from scipy.stats import ttest_rel

from sklearn.metrics import r2_score, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
warnings.simplefilter("ignore")
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
pd.set_option('display.max_columns', None)

In [2]:
DATA_PATH = './'
TRAIN_PATH = os.path.join(DATA_PATH, 'assignment_train.csv')
TEST_PATH = os.path.join(DATA_PATH, 'assignment_test.csv')
TARGET_COLUMN = 'isFraud'

## Задание 0

выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

**Комментрарий**

В задании 2 у меня лучше других показала себя схема кроссвалидации на 5 фолдов. Использованная модель: CatBoost

In [3]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [4]:
cat_features = train.select_dtypes("object").columns.tolist()

In [5]:
test[cat_features] = test[cat_features].astype(str)
train[cat_features] = train[cat_features].astype(str)

In [6]:
public_test_x = test.drop(["TransactionID", TARGET_COLUMN], axis=1)
public_test_y = test[TARGET_COLUMN]

In [7]:
config_name = "configs/catboost_config.yaml"
with open(config_name) as cfg:
        model_params = yaml.load(cfg)

In [8]:
model = cb.CatBoostClassifier(**model_params)

In [9]:
cv_strategy = KFold(n_splits=5, random_state=27)

estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    train.drop(columns=['TransactionID', TARGET_COLUMN]), train[TARGET_COLUMN], model, 
    metric=roc_auc_score, cv_strategy=cv_strategy, cat_features=cat_features
)

Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.8851, valid-score = 0.8384
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.8826, valid-score = 0.8569
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.8764, valid-score = 0.8811
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.8791, valid-score = 0.8597
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.8779, valid-score = 0.8856
CV-results train: 0.8802 +/- 0.003
CV-results valid: 0.8644 +/- 0.017
OOF-score = 0.8627


In [10]:
y_pred_lb = 0
for estimator in estimators:
    y_pred_lb += estimator.predict_proba(public_test_x)[:, 1]
    
y_pred_lb = y_pred_lb / 5

public_test_score = roc_auc_score(public_test_y, 
                                  y_pred_lb)

print(public_test_score)

BASE_OOF_SCORE = oof_score
BASE_PUBLIC_SCORE = public_test_score

0.8599287618923193


In [11]:
BASE_SCORES_FOR_TTEST = np.array([])

for i in range(10):
    cv_strategy = KFold(n_splits=5, random_state=i)

    estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
        train.drop(columns=['TransactionID', TARGET_COLUMN]), train[TARGET_COLUMN], model, 
        metric=roc_auc_score, cv_strategy=cv_strategy, cat_features=cat_features, log=False
    )
    
    BASE_SCORES_FOR_TTEST = np.append(BASE_SCORES_FOR_TTEST, fold_valid_scores)

**ВЫВОД**

Базовое качество модели при фиксированной схеме валидации: 0.8599

## Задание 1

признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [12]:
import copy
import datetime

In [13]:
def create_date_features(dataset, datetime_feature, start_date=None):
    """Функция создает в *dataset* признаки на основе даты:
    год, месяц, номер недели, день месяца, день недели, час 
    
    Input
    -----
    dataset: pd.Dataframe
        датасет, в который будут добавлены новые столбцы
    
    datetime_feature: str
        название признака, содержащего дату в формате timestamp в *dataset*
        
    start_date: datetime, optional
        начальная дата
        
    Output
    ------
    dataset: pd.Dataframe
        копия исходного датасета с добавленными признаками
    """
    
    if start_date:
        if not isinstance(start_date, datetime.datetime):
            msg = "invalid type for start_date"
            raise TypeError(msg)
            
    dataset = copy.deepcopy(dataset)
    
    if start_date:
        timedeltas = dataset[datetime_feature].apply(lambda x: datetime.timedelta(seconds=int(x)))
        actual_dates = timedeltas + start_date
        
    else:
        actual_dates = dataset[datetime_feature].apply(datetime.datetime.fromtimestamp())
        
    dataset["year"] = actual_dates.dt.year
    dataset["month"] = actual_dates.dt.month
    dataset["week_of_year"] = actual_dates.dt.weekofyear
    dataset["day_of_year"] = actual_dates.dt.dayofyear
    dataset["day_of_month"] = actual_dates.dt.day
    dataset["day_of_week"] = actual_dates.dt.weekday
    dataset["hour"] = actual_dates.dt.hour
    
    return dataset

In [14]:
start_date = datetime.datetime(2017, 12, 1)

In [15]:
train_wdates = create_date_features(train, 'TransactionDT', start_date)
public_test_x_wdates = create_date_features(public_test_x, 'TransactionDT', start_date)

In [16]:
train_wdates.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V337,V338,V339,year,month,week_of_year,day_of_year,day_of_month,day_of_week,hour
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,2017,12,48,336,2,5,0
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,2017,12,48,336,2,5,0
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,2017,12,48,336,2,5,0
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,2017,12,48,336,2,5,0
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,2017,12,48,336,2,5,0


In [17]:
estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    train_wdates.drop(columns=['TransactionID', TARGET_COLUMN]), train[TARGET_COLUMN], model, 
    metric=roc_auc_score, cv_strategy=cv_strategy, cat_features=cat_features
)

Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.889, valid-score = 0.8374
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.8807, valid-score = 0.8586
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.8782, valid-score = 0.883
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.8832, valid-score = 0.8636
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.88, valid-score = 0.8865
CV-results train: 0.8822 +/- 0.004
CV-results valid: 0.8658 +/- 0.018
OOF-score = 0.864


In [18]:
y_pred_lb = 0
for estimator in estimators:
    y_pred_lb += estimator.predict_proba(public_test_x_wdates)[:, 1]
    
y_pred_lb = y_pred_lb / 5

public_test_score = roc_auc_score(public_test_y, 
                                  y_pred_lb)
public_test_score

0.8610899625267636

In [19]:
scores = np.array([])

for i in range(10):
    cv_strategy = KFold(n_splits=5, random_state=i)

    estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
        train_wdates.drop(columns=['TransactionID', TARGET_COLUMN]), train[TARGET_COLUMN], model, 
        metric=roc_auc_score, cv_strategy=cv_strategy, cat_features=cat_features, log=False
    )
    
    scores = np.append(scores, fold_valid_scores)

In [20]:
print(f'OOF_SCORE изменился на {oof_score - BASE_OOF_SCORE:.4f}')
print(f'Public score изменился на {public_test_score - BASE_PUBLIC_SCORE:.4f}')

OOF_SCORE изменился на 0.0014
Public score изменился на 0.0012


In [21]:
ttest_rel(scores, BASE_SCORES_FOR_TTEST)

Ttest_relResult(statistic=6.4754572833311075, pvalue=4.310757210152398e-08)

**ВЫВОД**

с признаками даты результаты и на кроссвалидации и на паблик тесте стали незначительно лучше. Вместе с тем доверительные интервалы увеличились на 0.001. При исследовани датасета было показано, что транзакции имеют неярко выраженную сезонность, возможно, это и повлияло на улучшенние прогнозов. По значению t-теста также видно, что улучшение прогнозов статистически значимо

## Задание 2

сгруппировать данные по card1 и посчитать среднюю сумму транзакции. Добавить в качестве признака в набор данных. Посчитать разницу между суммой транзакцией пользователя и средней суммой транзакции по данному типу card1. Построить отношение этих признаков. Повторить процедуру для всех card.

In [22]:
all_data = pd.concat([train, test])

In [23]:
def create_numerical_aggs(data: pd.DataFrame,
                          groupby_id: str,
                          aggs: dict,
                          prefix: Optional[str] = None,
                          suffix: Optional[str] = None,
                          ) -> pd.DataFrame:
    """
    Построение агрегаций для числовых признаков.

    Parameters
    ----------
    data: pandas.core.frame.DataFrame
        Выборка для построения агрегаций.

    groupby_id: str
        Название ключа, по которому нужно произвести группировку.

    aggs: dict
        Словарь с названием признака и списка функций.
        Ключ словаря - название признака, который используется для
        вычисления агрегаций, значение словаря - список с названием
        функций для вычисления агрегаций.

    prefix: str, optional, default = None
        Префикс для названия признаков.
        Опциональный параметр, по умолчанию, не используется.

    suffix: str, optional, default = None
        Суффикс для названия признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    stats: pandas.core.frame.DataFrame
        Выборка с рассчитанными агрегациями.

    """
    if not prefix:
        prefix = ""
    if not suffix:
        suffix = ""

    data_grouped = data.groupby(groupby_id)
    stats = data_grouped.agg(aggs)
    stats.columns = [f"{prefix}{feature}_{stat}{suffix}".upper() for feature, stat in stats]
    stats = stats.reset_index()

    return stats

In [25]:
aggs = {'TransactionAmt': [np.mean]}

for feature in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6']:
    stats = create_numerical_aggs(all_data, groupby_id=feature, aggs=aggs)
    stat_name = stats.columns[-1]
    
    train_agg = train.merge(stats, how='left', on=feature)
    public_test_x_agg = public_test_x.merge(stats, how='left', on=feature)
    
    train_agg[f'TRANSACTIONAMT - {stat_name}'] = train_agg['TransactionAmt'] - train_agg[stat_name]
    public_test_x_agg[f'TRANSACTIONAMT - {stat_name}'] = public_test_x_agg['TransactionAmt'] -\
        public_test_x_agg[stat_name]
    
    train_agg[f'TRANSACTIONAMT / {stat_name}'] = train_agg['TransactionAmt'] / train_agg[stat_name]
    public_test_x_agg[f'TRANSACTIONAMT / {stat_name}'] = public_test_x_agg['TransactionAmt'] /\
        public_test_x_agg[stat_name]
    
    print('*'*100)
    print(feature.upper())
    print('*'*100)
    
    estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
        train_agg.drop(columns=['TransactionID', TARGET_COLUMN]), train_agg[TARGET_COLUMN], model, 
        metric=roc_auc_score, cv_strategy=cv_strategy, cat_features=cat_features
    )
    
    y_pred_lb = 0
    for estimator in estimators:
        y_pred_lb += estimator.predict_proba(public_test_x_agg)[:, 1]

    y_pred_lb = y_pred_lb / 5

    public_test_score = roc_auc_score(public_test_y, 
                                      y_pred_lb)
    
    scores = np.array([])

    for i in range(10):
        cv_strategy = KFold(n_splits=5, random_state=i)

        estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
            train_agg.drop(columns=['TransactionID', TARGET_COLUMN]), train_agg[TARGET_COLUMN], model, 
            metric=roc_auc_score, cv_strategy=cv_strategy, cat_features=cat_features, log=False
        )

        scores = np.append(scores, fold_valid_scores)
        
    print(f'Public test score: {public_test_score:.4f}')
    
    print(f'OOF_SCORE изменился на {oof_score - BASE_OOF_SCORE:.4f}')
    print(f'Public score изменился на {public_test_score - BASE_PUBLIC_SCORE:.4f}')
    print(f'Ttest: {ttest_rel(scores, BASE_SCORES_FOR_TTEST)}')

****************************************************************************************************
CARD1
****************************************************************************************************
Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.8866, valid-score = 0.8379
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.8801, valid-score = 0.8569
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.8773, valid-score = 0.8786
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.8825, valid-score = 0.8677
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.8779, valid-score = 0.8816
CV-results train: 0.8809 +/- 0.003
CV-results valid: 0.8645 +/- 0.016
OOF-score = 0.8627
Public test score: 0.8607
OOF_SCORE изменился на 0.0000
Public score изменился на 0.0007
Ttest: Ttest_relResult(statistic=0.3112118563765284, pvalue=0.7569597562868753)

**ВЫВОД**

Значимые изменения показали только новые признаки на основе признаков card3 и card4. Card3 показал значительные улучшения, случай card4 интереснее тем, что предсказание на тесте потеряло точность (но в пределах доверительного интервала), а вот значение статистики говорит о том, что вклад новых признаков положительный.

## Задание 3

преобразовать признаки card_1 - card_6 с помощью Frequency Encoding

In [ ]:
# Оформил в функцию сравнение двух моделей и вывод статистики
def compare_with_base(train, test, model, base_oof, base_public, base_ttest):
    cv_strategy = KFold(n_splits=5, random_state=27)
    
    estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
        train.drop(columns=['TransactionID', TARGET_COLUMN]), train[TARGET_COLUMN], model, 
        metric=roc_auc_score, cv_strategy=cv_strategy, cat_features=cat_features
    )

    y_pred_lb = 0
    for estimator in estimators:
        y_pred_lb += estimator.predict_proba(test)[:, 1]

    y_pred_lb = y_pred_lb / 5

    public_test_score = roc_auc_score(public_test_y, 
                                      y_pred_lb)

    scores = np.array([])

    for i in range(10):
        cv_strategy = KFold(n_splits=5, random_state=i)

        estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
            train.drop(columns=['TransactionID', TARGET_COLUMN]), train[TARGET_COLUMN], model, 
            metric=roc_auc_score, cv_strategy=cv_strategy, cat_features=cat_features, log=False
        )

        scores = np.append(scores, fold_valid_scores)

    print(f'Public test score: {public_test_score:.4f}')

    print(f'OOF_SCORE изменился на {oof_score - BASE_OOF_SCORE:.4f}')
    print(f'Public score изменился на {public_test_score - BASE_PUBLIC_SCORE:.4f}')
    print(f'Ttest: {ttest_rel(scores, BASE_SCORES_FOR_TTEST)}')

In [33]:
train_freq = copy.deepcopy(train)
public_test_x_freq = copy.deepcopy(public_test_x)

for feature in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6']:
    freq_encoder = all_data[feature].value_counts(normalize=True)
    train_freq[f'{feature}_freq_enc'] = train_freq[feature].map(freq_encoder)
    public_test_x_freq[f'{feature}_freq_enc'] = public_test_x_freq[feature].map(freq_encoder)
    
    train_freq.drop(columns=[feature])
    public_test_x_freq.drop(columns=[feature])
    
    print('*'*100)
    print(feature.upper())
    print('*'*100)
    
    compare_with_base(train_freq, public_test_x_freq, model, BASE_OOF_SCORE, 
                      BASE_PUBLIC_SCORE, BASE_SCORES_FOR_TTEST)

****************************************************************************************************
CARD1
****************************************************************************************************
Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.888, valid-score = 0.8386
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.8796, valid-score = 0.8569
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.8769, valid-score = 0.8823
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.882, valid-score = 0.8607
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.879, valid-score = 0.886
CV-results train: 0.8811 +/- 0.004
CV-results valid: 0.8649 +/- 0.017
OOF-score = 0.8632
Public test score: 0.8616
OOF_SCORE изменился на 0.0005
Public score изменился на 0.0016
Ttest: Ttest_relResult(statistic=8.637651497634545, pvalue=2.0577869071666217e-11)


**ВЫВОД**

Значимые изменения показали преобразования признаков CARD1, CARD3, CARD4 и CARD6. Но в случае с CARD6 прогнозы ухудшились, поэтому это преобразование не считается удачным.

## Задание 4

преобразовать признак TransactionAmt в логариф признака, выделить дробную часть и целую часть в отдельные признаки

In [83]:
train_log = copy.deepcopy(train)
public_test_x_log = copy.deepcopy(public_test_x)

for data in (train_log, public_test_x_log):
    data['TransactionAmt'] = data['TransactionAmt'].apply(np.log)
    data.rename(columns={'TransactionAmt': 'LogTransAmt'}, inplace=True)
    data['LogTransAmt_frac'] = data['LogTransAmt'].map(lambda x: np.modf(x)[0])
    data['LogTransAmt_int'] = data['LogTransAmt'].map(lambda x: np.modf(x)[1])
    
compare_with_base(train_log, public_test_x_log, model, BASE_OOF_SCORE, 
                  BASE_PUBLIC_SCORE, BASE_SCORES_FOR_TTEST)

Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.882, valid-score = 0.8336
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.8804, valid-score = 0.8577
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.8752, valid-score = 0.8766
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.8782, valid-score = 0.8606
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.879, valid-score = 0.8879
CV-results train: 0.879 +/- 0.002
CV-results valid: 0.8633 +/- 0.018
OOF-score = 0.8614
Public test score: 0.8590
OOF_SCORE изменился на -0.0013
Public score изменился на -0.0009
Ttest: Ttest_relResult(statistic=-2.499424241475067, pvalue=0.01583853157848215)


**ВЫВОД**

Нулевая гипотеза о равенстве распределений не может быть отклонена, качество модели после преобразования не изменилось. Переходя к логарифму признака, мы рассчитываем сделать данные более нормально распределенными, но, судя по всему, для деревяннных моделей распределение не очень важно.

## Задание 5

для числовых признаков построить PCA-признаки, добавить их к основной части датасета.

**Комментарий**

В EDA было показано наличие большого количества пропусков в датасете. До этого момента пропуски никак не обрабатывались, а PCA не умеет автоматически с ними справляться. Поэтому применю PCA только для полностью заполненных числовых колонок. Кроме того перед применением PCA необходимо нормализовать данные.

In [84]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

columns_wo_nan = all_data[all_data.columns[~all_data.isnull().any()]].columns.tolist()
numerical = train[columns_wo_nan].drop(columns=['TransactionID', TARGET_COLUMN])\
    .select_dtypes([np.number]).columns.tolist()

scaler = StandardScaler()
all_data_scaled = scaler.fit_transform(all_data[numerical])

pca = PCA(n_components=len(numerical))
pca.fit(all_data_scaled)

np.cumsum(pca.explained_variance_ratio_)

array([0.15191777, 0.29229266, 0.40461155, 0.47346957, 0.52940315,
       0.5712165 , 0.61146674, 0.6483397 , 0.68041327, 0.70752409,
       0.73002076, 0.75127533, 0.7698987 , 0.78685628, 0.80232055,
       0.8170429 , 0.83117101, 0.84487214, 0.85797881, 0.87075058,
       0.88278623, 0.89346898, 0.90268448, 0.9116765 , 0.9204427 ,
       0.92909047, 0.93699095, 0.94375905, 0.94975542, 0.95532819,
       0.96037617, 0.96446714, 0.96830648, 0.97172987, 0.9750716 ,
       0.97782612, 0.98032974, 0.98244956, 0.98446487, 0.9861024 ,
       0.98757124, 0.98899398, 0.99032984, 0.99160604, 0.99282291,
       0.99401429, 0.99518718, 0.99604816, 0.99663262, 0.99717724,
       0.99768175, 0.99812333, 0.99853717, 0.99892043, 0.99926331,
       0.99950245, 0.99969771, 0.99983096, 0.99988913, 0.99992084,
       0.99994244, 0.99996101, 0.99997337, 0.99998399, 0.99999149,
       0.99999644, 0.99999806, 0.99999957, 0.99999986, 0.99999999,
       1.        , 1.        ])

In [86]:
train_pca = copy.deepcopy(train)
public_test_x_pca = copy.deepcopy(public_test_x)

data_scaled = scaler.transform(train_pca[numerical])
transformed_components = pca.transform(data_scaled)
pca_df = pd.DataFrame(data=transformed_components)
train_pca = train_pca.join(pca_df)

data_scaled = scaler.transform(public_test_x_pca[numerical])
transformed_components = pca.transform(data_scaled)
pca_df = pd.DataFrame(data=transformed_components)
public_test_x_pca = public_test_x_pca.join(pca_df)

compare_with_base(train_pca, public_test_x_pca, model, BASE_OOF_SCORE, 
                  BASE_PUBLIC_SCORE, BASE_SCORES_FOR_TTEST)

Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.8884, valid-score = 0.8395
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.8846, valid-score = 0.8568
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.8774, valid-score = 0.8862
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.8821, valid-score = 0.8613
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.8822, valid-score = 0.8856
CV-results train: 0.8829 +/- 0.004
CV-results valid: 0.8659 +/- 0.018
OOF-score = 0.8639
Public test score: 0.8630
OOF_SCORE изменился на 0.0012
Public score изменился на 0.0031
Ttest: Ttest_relResult(statistic=5.6695947515979075, pvalue=7.525307308400315e-07)


**ВЫВОД**

PCA признаки дают хороший значимый прирост

## Заданние 6

использовать критерий отбора признаков на основе перестановок для отбора признаков, которые положительно влияют на перформанс модели. Переобучить модель и сделать выводы о полученном качестве алгоритма.

**Комментарий**

Так как в процессе иследования преобразования в датасете отбирались на основе ttest, интересно сравнить качество модели только на "хороших" дополнительных признаках с качеством на всех признаках. После этого применю метод пермутированной важности и постараюсь отобрать признаки, чтобы еще больше повысить качество модели

In [131]:
def add_new_features(data_train: pd.DataFrame, data_test: pd.DataFrame, 
                     start_date: datetime.datetime, 
                     card_features: List, freq_features: List,
                     log_transform: bool
                     ):
    data_train = copy.deepcopy(data_train)
    data_test = copy.deepcopy(data_test)
    
    # признаки на основе даты
    data_train = create_date_features(data_train, 'TransactionDT', start_date)
    data_test = create_date_features(data_test, 'TransactionDT', start_date)
    
    # Скомбинированные по данным карты признаки
    all_data = pd.concat([data_train, data_test])
    aggs = {'TransactionAmt': [np.mean]}
    
    for feature in card_features:
        stats = create_numerical_aggs(all_data, groupby_id=feature, aggs=aggs, suffix=f"_ON_{feature}")
        stat_name = stats.columns[-1]

        data_train = data_train.merge(stats, how='left', on=feature) 
        data_train[f'TRANSACTIONAMT - {stat_name}'] = data_train['TransactionAmt'] - data_train[stat_name]
        data_train[f'TRANSACTIONAMT / {stat_name}'] = data_train['TransactionAmt'] / data_train[stat_name]
        
        data_test = data_test.merge(stats, how='left', on=feature) 
        data_test[f'TRANSACTIONAMT - {stat_name}'] = data_test['TransactionAmt'] - data_test[stat_name]
        data_test[f'TRANSACTIONAMT / {stat_name}'] = data_test['TransactionAmt'] / data_test[stat_name]
    
    # Признаки закодированные с помощью частот
    all_data = pd.concat([data_train, data_test])
    
    for feature in freq_features:
        for data in [data_train, data_test]:
            freq_encoder = all_data[feature].value_counts(normalize=True)
            data[f'{feature}_freq_enc'] = data[feature].map(freq_encoder)
            data.drop(columns=[feature], inplace=True)
    
    # преобразование в логарифм признака TransactionAmt
    if log_transform:
        for data in [data_train, data_test]:
            data['TransactionAmt'] = data['TransactionAmt'].apply(np.log)
            data.rename(columns={'TransactionAmt': 'LogTransAmt'}, inplace=True)
            data['LogTransAmt_frac'] = data['LogTransAmt'].map(lambda x: np.modf(x)[0])
            data['LogTransAmt_int'] = data['LogTransAmt'].map(lambda x: np.modf(x)[1])
        
    # PCA признаки
    all_data = pd.concat([data_train, data_test])
    columns_wo_nan = all_data[all_data.columns[~all_data.isnull().any()]].columns.tolist()
    numerical = all_data[columns_wo_nan].select_dtypes([np.number]).columns.tolist()
    
    scaler = StandardScaler()
    all_data_scaled = scaler.fit_transform(all_data[numerical])

    pca = PCA(n_components=len(numerical))
    pca.fit(all_data_scaled)
    
    data_scaled = scaler.transform(data_train[numerical])
    transformed_components = pca.transform(data_scaled)
    pca_df = pd.DataFrame(data=transformed_components)
    data_train = data_train.join(pca_df)
    
    data_scaled = scaler.transform(data_test[numerical])
    transformed_components = pca.transform(data_scaled)
    pca_df = pd.DataFrame(data=transformed_components)
    data_test = data_test.join(pca_df)
    
    return data_train, data_test

In [132]:
card_features_all = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6']
freq_features_all = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6']

card_features_best = ['card3', 'card4']
freq_features_best = ['card1', 'card3', 'card4']

In [136]:
train_all, public_test_x_all = add_new_features(train, public_test_x, start_date, 
                                                card_features_all, freq_features_all,
                                                log_transform=True)

train_best, public_test_x_best = add_new_features(train, public_test_x, start_date, 
                                                  card_features_best, freq_features_best,
                                                  log_transform=False)

In [137]:
compare_with_base(train_all, public_test_x_all, model, BASE_OOF_SCORE, 
                  BASE_PUBLIC_SCORE, BASE_SCORES_FOR_TTEST)

Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.8935, valid-score = 0.84
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.887, valid-score = 0.8612
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.8795, valid-score = 0.8835
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.8862, valid-score = 0.8635
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.8843, valid-score = 0.8869
CV-results train: 0.8861 +/- 0.005
CV-results valid: 0.867 +/- 0.017
OOF-score = 0.8651
Public test score: 0.8653
OOF_SCORE изменился на 0.0024
Public score изменился на 0.0054
Ttest: Ttest_relResult(statistic=15.73690525026213, pvalue=8.550084992622942e-21)


In [138]:
compare_with_base(train_best, public_test_x_best, model, BASE_OOF_SCORE, 
                  BASE_PUBLIC_SCORE, BASE_SCORES_FOR_TTEST)

Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.8897, valid-score = 0.8411
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.8817, valid-score = 0.8551
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.8769, valid-score = 0.8818
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.8837, valid-score = 0.8631
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.8839, valid-score = 0.885
CV-results train: 0.8832 +/- 0.004
CV-results valid: 0.8652 +/- 0.016
OOF-score = 0.863
Public test score: 0.8656
OOF_SCORE изменился на 0.0003
Public score изменился на 0.0057
Ttest: Ttest_relResult(statistic=3.128037594541308, pvalue=0.00296033306367832)


**Комментарий**

Получилось интересно. Обе модели дают значимый прирост, но при этом, согласно ttest, предсказания первой модели от базовой отличаются амного сильнее. Вторая модель показала незначительно лучший скор на паблике, но при этом меньшее улучшение на OOF выборке. Для дальнейшего исследования и расчета пермутированной важности воспользуюсь данными с полным набором измененных признаков. 

In [143]:
def calculate_permutation_importance(estimator, 
                                     metric: callable,
                                     x_valid: pd.DataFrame,
                                     y_valid: pd.Series) -> pd.Series:
    """
    Расчет пермутированной важности признаков.
    """
    scores = {}
    y_pred = estimator.predict_proba(x_valid)[:, 1]
    base_score = metric(y_valid, y_pred)

    for feature in tqdm(x_valid.columns):
        x_valid_copy = x_valid.copy()
        x_valid_copy[feature] = np.random.permutation(x_valid_copy[feature])

        y_pred = estimator.predict_proba(x_valid_copy)[:, 1]
        score = metric(y_valid, y_pred)
        scores[feature] = base_score - score

    scores = pd.Series(scores)
    scores = scores.sort_values(ascending=False)

    return scores

In [146]:
cat_features = train_all.select_dtypes("object").columns.tolist()
train_all[cat_features] = train_all[cat_features].astype(str)

In [147]:
x_train, x_valid = train_test_split(
    train_all.drop(columns=['TransactionID', TARGET_COLUMN], axis=1), train_size=0.7, random_state=1
)
y_train, y_valid = train_test_split(
    train_all[TARGET_COLUMN], train_size=0.7, random_state=1
)

In [148]:
model.fit(x_train, y_train, cat_features=cat_features, verbose=0, 
                      eval_set=(x_valid, y_valid))

In [149]:
perm_importance = calculate_permutation_importance(
    estimator=model, metric=roc_auc_score, x_valid=x_valid, y_valid=y_valid
)

100%|██████████| 511/511 [01:07<00:00,  7.53it/s]


In [151]:
bad_features = [feature for feature in perm_importance.keys() if perm_importance[feature] < 0]

In [153]:
perm_importance['day_of_week']

-5.002926712904809e-07

In [160]:
train_filtered = train_all.drop(columns=bad_features)
public_test_x_filtered = public_test_x_all.drop(columns=bad_features)

compare_with_base(train_filtered, public_test_x_filtered, model, BASE_OOF_SCORE, 
                  BASE_PUBLIC_SCORE, BASE_SCORES_FOR_TTEST)

Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.8912, valid-score = 0.8419
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.8847, valid-score = 0.858
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.8825, valid-score = 0.8844
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.8879, valid-score = 0.8682
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.8835, valid-score = 0.8871
CV-results train: 0.886 +/- 0.003
CV-results valid: 0.8679 +/- 0.017
OOF-score = 0.866
Public test score: 0.8655
OOF_SCORE изменился на 0.0033
Public score изменился на 0.0056
Ttest: Ttest_relResult(statistic=9.485266190050506, pvalue=1.1290934571639186e-12)


**ВЫВОД**

после отбрасывания неважных признаков, скор на паблике упал на 0.01%, измененения ввсе так же значимы.